# Reproduce Ifeoma's APAC_shrink_append.do script

Get file information

In [1]:
path <- "//CHSE/DataRepository/APAC/Raw/LimitedDatasets/Release20150902"
f <- grep("episodes.*201[0-3]\\.txt", list.files(path), ignore.case=TRUE, value=TRUE)
f <- file.path(path, f)
file.info(f)[c("size", "mtime")]

,size,mtime
//CHSE/DataRepository/APAC/Raw/LimitedDatasets/Release20150902/Mcconn_episodes2010.txt,30869374063,2015-08-31 15:27:02
//CHSE/DataRepository/APAC/Raw/LimitedDatasets/Release20150902/Mcconn_episodes2011.txt,43410281308,2015-08-31 16:30:25
//CHSE/DataRepository/APAC/Raw/LimitedDatasets/Release20150902/Mcconn_episodes2012.txt,45270624481,2015-08-31 18:58:27
//CHSE/DataRepository/APAC/Raw/LimitedDatasets/Release20150902/Mcconn_episodes2013.txt,45723795636,2015-09-01 14:35:55


Specify file characteristics

* Column names
* Column classes
* Columns to select
* NA strings

In [2]:
library(data.table)
varnames <- names(fread(f[1], nrows=0))
names(varnames) <- 1:length(varnames)
select <- c(7, 13, 14, 15, 16, 23, 24, 35, 36:48, 62:74, 75:79, 98)
colClasses <- rep("character", length(varnames))
colClasses[varnames %in% c("patid", "personkey", "yob", "megnum")] <- "integer"
names(colClasses) <- varnames
na.strings <- c("NA", "*N", "*NU", "*NUL", "*NULL", "*NULL*", "")

Parallelize the file reads

In [3]:
library("doParallel")
years <- length(f)
cl <- makeCluster(years)
registerDoParallel(cl, cores=years)
L <- foreach (i = 1:years) %dopar% {
    require(data.table)
    Di <- fread(f[i], nrow=1e5, select=select, colClasses=colClasses, na.strings=na.strings)
    Di <- Di[gender != "M" & between(yob, 1955 + (i - 1), 2000 + (i - 1))]
    Di <- Di[,
             `:=` (fromdate = as.Date(fromdate),
                   todate = as.Date(todate))]
    Di
}
stopCluster(cl)
D <- rbindlist(L)
D[, .(.N, minYOB = min(yob), maxYOB = max(yob)), .(year(fromdate), gender)]

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


,year,gender,N,minYOB,maxYOB
1,2010,F,31257,1955,2000
2,2011,F,7605,1956,2001
3,2012,F,24186,1957,2002
4,2013,F,12366,1958,2003


Write to Stata `.dta` file

In [4]:
library(haven)
path <- "F:/OD11ObGynHardStop/Data"
f <- file.path(path, "APAC_episodes_childbearingwomen.dta")
write_dta(D, f)
file.info(f)[c("size", "mtime")]
message(sprintf("Size of data.table() object D is %.3g GB", object.size(D) / 1e9))
str(D)

,size,mtime
F:/OD11ObGynHardStop/Data/APAC_episodes_childbearingwomen.dta,11997422,2016-05-20 06:50:15


Size of data.table() object D is 0.0236 GB


Classes 'data.table' and 'data.frame':	75414 obs. of  40 variables:
 $ apac_payer: chr  "MCO" "MCO" "MCO" "MCO" ...
 $ patid     : int  2336687 3998961 3998961 3998961 3998961 4107598 3024345 1911465 3998961 1959472 ...
 $ personkey : int  7892947 7251101 7251101 7251101 7251101 1798306 4164607 7258740 7251101 7243618 ...
 $ gender    : chr  "F" "F" "F" "F" ...
 $ yob       : int  1999 1986 1986 1986 1986 1994 1985 2000 1986 1963 ...
 $ fromdate  : Date, format: "2010-01-07" "2010-04-20" ...
 $ todate    : Date, format: "2010-01-07" "2010-04-20" ...
 $ hcg       : chr  "P99c" "P99c" "P99c" "P99c" ...
 $ dx1       : chr  NA NA NA NA ...
 $ dx2       : chr  NA NA NA NA ...
 $ dx3       : chr  NA NA NA NA ...
 $ dx4       : chr  NA NA NA NA ...
 $ dx5       : chr  NA NA NA NA ...
 $ dx6       : chr  NA NA NA NA ...
 $ dx7       : chr  NA NA NA NA ...
 $ dx8       : chr  NA NA NA NA ...
 $ dx9       : chr  NA NA NA NA ...
 $ dx10      : chr  NA NA NA NA ...
 $ dx11      : chr  NA NA NA NA 